In [1]:
import torch

In [2]:
torch.__version__

'2.4.1'

In [5]:
torch.cuda.is_available()

True

In [10]:
torch.cuda.get_device_name()

'NVIDIA L40S'

In [11]:
torch.cuda.list_gpu_processes()

'GPU:0\nno processes are running'

In [12]:
torch.cuda.device_count()

1

In [19]:
# See https://github.com/seafoodfry/ml-workspace/blob/main/gpu-sandbox/our-cuda-by-example/03-cudaDeviceProp.cu
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA L40S', major=8, minor=9, total_memory=45372MB, multi_processor_count=142)

In [18]:
# See https://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/
torch.cuda.get_arch_list()

['sm_50', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90']

In [20]:
# See
# https://pytorch.org/docs/stable/notes/cuda.html#optimizing-memory-usage-with-pytorch-cuda-alloc-conf
# and
# https://developer.nvidia.com/blog/using-cuda-stream-ordered-memory-allocator-part-1/
torch.cuda.get_allocator_backend()

'native'

In [24]:
# See https://pytorch.org/docs/stable/notes/mps.html
torch.backends.mps.is_available()

False

In [25]:
torch.backends.cuda.is_built()

True

In [26]:
torch.backends.cpu.get_cpu_capability()

'AVX2'

In [30]:
torch.tensor([1])

tensor([1])

In [31]:
torch.tensor([1, 2, 3])

tensor([1, 2, 3])

In [34]:
torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
])

tensor([[1, 2, 3],
        [4, 5, 6]])

In [36]:
# 3D.
torch.tensor([
    [
        [1, 2, 3],
        [4, 5, 6],
    ],
    [
        [7, 8, 9],
        [10, 11, 12],
    ]
])

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]]])

In [37]:
torch.tensor([1]).dtype

torch.int64

In [39]:
torch.tensor([1]).to(torch.float32).dtype

torch.float32

In [38]:
torch.tensor([1.0]).dtype

torch.float32